## Test Result

<mark>This notebook is called from an individual test case notebook; before you run this notebook, please `xmltodict` installed.</mark>  

In [ ]:
from pyspark.sql.functions import when,col, explode,isnull,from_json
from pyspark.sql.types import StructType,StructField,StringType
import json
import xmltodict

# convert xml to dict / json
dict_result=xmltodict.parse(out.getvalue())
json_result = json.loads(json.dumps(dict_result,indent=4).replace('@',''))
test_suites=json_result['testsuites']['testsuite']

# create dataframe
df = spark.read.json(sc.parallelize([test_suites]))

fail_schema = StructType([
  StructField("command", StringType(), True),
  StructField("status", StringType(),  True)
])

explode_df= df.withColumn('ts',explode('testcase')).withColumn('fail',from_json(col('ts.failure.message'),fail_schema)).drop(col('testcase')).drop(col('ts.failure'))

df_test_result=explode_df.select(col("errors").alias("errorInSuite"),col("failures").alias("failedInSuite"),col("name").alias("suitename"),"skipped",col("tests").alias("totalTest"),col("timestamp").alias("executionTime"),col("ts.name").alias("testCaseName"),col("ts.time").alias("testCaseTime"),col("fail.command"))

if (len(storage_account)>0 and len(result_container)>0):
    # save result to storage
    storage_path = f"abfs://{result_container}@{storage_account}.dfs.core.windows.net/{TEST_RUN_ID}/{PLATFORM}/{TEST_SUITE}"
    # write raw results
    df.write.parquet(f"{storage_path}/{RAW_RESULT_FILE_NAME}")
    # write transformed results
    df_test_result.write.parquet(f"{storage_path}/{RESULT_FILE_NAME}") 
else:
    print("configure storage path to store results")
    df_test_result.show(200,False)   